# SETUP

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
#importing he necessary libraries
from nltk.corpus import stopwords
# import nltk.tokenize as tokenizer
from nltk.tokenize import word_tokenize
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import RegexpTokenizer as regextoken
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#reading the csv data into a dataframe
train = pd.read_csv('/content/drive/MyDrive/Identify the sentiments/train_2kmZucJ.csv')
test = pd.read_csv('/content/drive/MyDrive/Identify the sentiments/test_oJQbWVk.csv')

# Checking for information about the datasets

In [5]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [7]:
train['label'].unique()

array([0, 1])

In [8]:
train.info,test.info

(<bound method DataFrame.info of         id  label                                              tweet
 0        1      0  #fingerprint #Pregnancy Test https://goo.gl/h1...
 1        2      0  Finally a transparant silicon case ^^ Thanks t...
 2        3      0  We love this! Would you go? #talk #makememorie...
 3        4      0  I'm wired I know I'm George I was made that wa...
 4        5      1  What amazing service! Apple won't even talk to...
 ...    ...    ...                                                ...
 7915  7916      0  Live out loud #lol #liveoutloud #selfie #smile...
 7916  7917      0  We would like to wish you an amazing day! Make...
 7917  7918      0  Helping my lovely 90 year old neighbor with he...
 7918  7919      0  Finally got my #smart #pocket #wifi stay conne...
 7919  7920      0  Apple Barcelona!!! #Apple #Store #BCN #Barcelo...
 
 [7920 rows x 3 columns]>,
 <bound method DataFrame.info of         id                                              tweet
 0  

# Text cleaning and preprocessing begins

In [9]:
def text_preprocessing(sentence):
  # convert to all text into lower case
  sentence = sentence.lower()
  # remove website links
  weblink_pattern = r'(\w+:\/\/\S+)'
  sentence = re.sub(pattern=weblink_pattern, repl=' ', string=sentence)
  # pattern to remove tagging usernames
  name_tag_pattern = r'(@[A-Za-z0-9]+)'
  sentence = re.sub(pattern=name_tag_pattern,repl=' ', string=sentence)
  # remove punctuations 
  punct_pattern = r'[-.#?()\/&^]+'
  sentence = re.sub(pattern=punct_pattern, repl=' ', string=sentence)
  # remove stop words 
  sentence = " ".join([word for word in str(sentence).split() if word not in stop_words])
  #remove extra spaces
  space_pattern = r'\s+'
  sentence = re.sub(pattern=space_pattern, repl=' ', string=sentence)
  # remove tabs 
  tab_pattern = r'\t+'
  sentence = re.sub(pattern=tab_pattern, repl=' ', string=sentence)
  # remove new lines
  line_pattern = r'\n+'
  sentence = re.sub(pattern=line_pattern, repl=' ', string=sentence)
  return sentence


In [10]:
train["tweet"] = train.tweet.apply(lambda x: text_preprocessing(x))
test["tweet"] = test.tweet.apply(lambda x: text_preprocessing(x))

In [11]:
# Preprocess the text with function processtext()
stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
lemma = WordNetLemmatizer() # for Lemmatisation


def processtext(text):
    text=re.sub("[^a-zA-Z]"," ",text) # Filter to allow only alphabets in text
    text=text.lower() # Convert the text to lowercase to maintain consistency
    tokens=word_tokenize(text) # Tokenize the text
    # Remove stopwords and punctuations
    tokens=[token for token in tokens if token not in stop_words and token not in punctuations] 
    tokens=[lemma.lemmatize(token) for token in tokens] # Lemmatisation of tokens
    text=" ".join(tokens)
    return text

In [12]:
train["tweet"] = train.tweet.apply(lambda x: processtext(x))
test["tweet"] = test.tweet.apply(lambda x: processtext(x))

In [23]:
train['label'].unique()

array([0, 1])

In [14]:
test.head()

,id,tweet
0,7921,hate new iphone upgrade let download apps ugh ...
1,7922,currently shitting fucking pant apple imac cas...
2,7923,like put cd rom ipad possible yes block screen
3,7924,ipod officially dead lost picture video so con...
4,7925,fighting itunes night want music paid


In [16]:
tokenizer = Tokenizer(num_words=1500, split=' ')
tokenizer.fit_on_texts(train['tweet'].values)
X = tokenizer.texts_to_sequences(train['tweet'])
X = pad_sequences(X)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(train['tweet'],train['label'], test_size = 0.15, random_state = 42)

In [28]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

# Model training and evaluation

In [33]:
# model initialization
model =Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(24, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(train_padded, y_train,validation_data = (test_padded,y_test),epochs = 10, batch_size=32, verbose=1)

Epoch 1/10
211/211 [==============================] - 44s 193ms/step - loss: 0.3612 - accuracy: 0.8345 - val_loss: 0.2513 - val_accuracy: 0.8965
Epoch 2/10
211/211 [==============================] - 38s 181ms/step - loss: 0.2066 - accuracy: 0.9138 - val_loss: 0.2388 - val_accuracy: 0.9066
Epoch 3/10
211/211 [==============================] - 38s 179ms/step - loss: 0.1664 - accuracy: 0.9345 - val_loss: 0.2912 - val_accuracy: 0.8939
Epoch 4/10
211/211 [==============================] - 39s 185ms/step - loss: 0.1404 - accuracy: 0.9477 - val_loss: 0.2997 - val_accuracy: 0.8864
Epoch 5/10
211/211 [==============================] - 38s 179ms/step - loss: 0.1175 - accuracy: 0.9568 - val_loss: 0.3242 - val_accuracy: 0.8847
Epoch 6/10
211/211 [==============================] - 39s 187ms/step - loss: 0.0908 - accuracy: 0.9679 - val_loss: 0.3887 - val_accuracy: 0.8704
Epoch 7/10
211/211 [==============================] - 38s 180ms/step - loss: 0.0714 - accuracy: 0.9734 - val_loss: 0.4350 - val_ac

In [40]:
test_sequences = tokenizer.texts_to_sequences(test["tweet"])
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
# print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

test['labels'] = pred_labels
test.head(15)



,id,tweet,labels
0,7921,hate new iphone upgrade let download apps ugh ...,1
1,7922,currently shitting fucking pant apple imac cas...,1
2,7923,like put cd rom ipad possible yes block screen,1
3,7924,ipod officially dead lost picture video so con...,1
4,7925,fighting itunes night want music paid,1
5,7926,repost repostapp announce apple bourbon,0
6,7927,new apple software update really thing phone b...,1
7,7928,baby iphone iphone gold new apple appleisbest ...,0
8,7929,confused take time set appointment still waiti...,1
9,7930,fruit taste better pick apple healthy fruit ny...,0
